In [ ]:
# Notebook 3: Test RAG Pipeline
# Ce notebook teste le pipeline complet RAG

# Cellule 1: Imports
import sys
sys.path.append('../src')

from modules.ingestion import DocumentIngestion
from modules.chunking import TextChunker
from modules.embeddings import EmbeddingModel
from modules.retrieval import FAISSRetriever
from modules.generation import ResponseGenerator
from modules.config import config
from pathlib import Path

print("✅ Tous les modules importés")

# Cellule 2: Initialiser tous les composants
print("🚀 Initialisation du pipeline RAG...")

ingestion = DocumentIngestion()
chunker = TextChunker()
retriever = FAISSRetriever()
generator = ResponseGenerator()

print("✅ Pipeline initialisé")

# Cellule 3: Fonction helper pour le pipeline complet
def process_document_full_pipeline(file_path: Path):
    """
    Pipeline complet : ingestion → chunking → embedding → indexation
    """
    print(f"\n📄 Traitement de: {file_path.name}")
    
    # 1. Ingestion
    print("1️⃣ Extraction du texte...")
    doc_info = ingestion.process_document(file_path)
    print(f"   ✅ {doc_info['num_characters']} caractères extraits")
    
    # 2. Chunking
    print("2️⃣ Création des chunks...")
    chunks = chunker.create_chunks_with_metadata(
        doc_info['content'],
        doc_info['filename']
    )
    print(f"   ✅ {len(chunks)} chunks créés")
    
    # 3. Embeddings
    print("3️⃣ Génération des embeddings...")
    chunk_texts = [c['content'] for c in chunks]
    embeddings = retriever.embedding_model.encode(chunk_texts)
    print(f"   ✅ Embeddings: {embeddings.shape}")
    
    # 4. Indexation
    print("4️⃣ Ajout à l'index FAISS...")
    retriever.add_to_index(embeddings, chunks)
    print(f"   ✅ Index: {retriever.index.ntotal} vecteurs")
    
    return {
        'doc_info': doc_info,
        'chunks': chunks,
        'num_vectors': retriever.index.ntotal
    }

# Cellule 4: Traiter un ou plusieurs documents
documents_dir = config.DOCUMENTS_DIR
pdf_files = list(documents_dir.glob("*.pdf"))
txt_files = list(documents_dir.glob("*.txt"))

all_files = pdf_files + txt_files

if all_files:
    print(f"\n📚 {len(all_files)} document(s) trouvé(s)")
    
    for file_path in all_files[:3]:  # Limiter à 3 pour le test
        try:
            result = process_document_full_pipeline(file_path)
            print(f"✅ {file_path.name} traité avec succès")
        except Exception as e:
            print(f"❌ Erreur avec {file_path.name}: {e}")
    
    # Sauvegarder l'index
    retriever.save_index()
    print(f"\n💾 Index sauvegardé")
else:
    print("⚠️ Aucun document trouvé dans data/documents/")

# Cellule 5: Fonction pour tester une question
def ask_question(question: str, top_k: int = 5, verbose: bool = True):
    """
    Pose une question au système RAG
    """
    if verbose:
        print(f"\n❓ Question: {question}")
        print("=" * 80)
    
    # 1. Recherche
    if verbose:
        print("\n🔍 Recherche des passages pertinents...")
    retrieved_chunks = retriever.search(question, top_k=top_k)
    
    if verbose:
        print(f"✅ {len(retrieved_chunks)} chunks récupérés")
        print("\n📄 Top 3 passages:")
        for i, chunk in enumerate(retrieved_chunks[:3], 1):
            print(f"\n{i}. Score: {chunk['score']:.4f}")
            print(f"   Document: {chunk['document_name']}")
            print(f"   Extrait: {chunk['content'][:200]}...")
    
    # 2. Génération
    if verbose:
        print("\n🤖 Génération de la réponse...")
    response = generator.generate_answer(question, retrieved_chunks)
    
    if verbose:
        print("\n💡 RÉPONSE:")
        print("=" * 80)
        print(response['answer'])
        print("=" * 80)
        print(f"\n📚 Sources: {response['context_used']} chunks utilisés")
    
    return response

# Cellule 6: Tests avec différentes questions
if retriever.index and retriever.index.ntotal > 0:
    questions = [
        "Quel est le sujet principal du document ?",
        "Quels sont les points clés abordés ?",
        "Y a-t-il des dates importantes mentionnées ?",
        "Qui sont les personnes citées ?",
        "Quelles sont les conclusions ?"
    ]
    
    print("\n" + "=" * 80)
    print("🧪 TEST DU PIPELINE RAG")
    print("=" * 80)
    
    for question in questions[:2]:  # Tester 2 questions
        response = ask_question(question)
        print("\n" + "-" * 80 + "\n")
else:
    print("⚠️ Aucun document indexé. Exécutez d'abord la cellule 4.")

# Cellule 7: Évaluation de la pertinence
def evaluate_retrieval(question: str, top_k: int = 10):
    """
    Évalue la qualité de la recherche
    """
    results = retriever.search(question, top_k=top_k)
    
    scores = [r['score'] for r in results]
    
    print(f"\n📊 Analyse pour: '{question}'")
    print(f"Nombre de résultats: {len(results)}")
    print(f"Score moyen: {sum(scores)/len(scores):.4f}")
    print(f"Score max: {max(scores):.4f}")
    print(f"Score min: {min(scores):.4f}")
    
    # Afficher les scores
    import matplotlib.pyplot as plt
    
    plt.figure(figsize=(10, 5))
    plt.bar(range(len(scores)), scores)
    plt.xlabel("Rang du résultat")
    plt.ylabel("Score de similarité")
    plt.title(f"Distribution des scores de pertinence\n'{question}'")
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    return results

# Cellule 8: Test d'évaluation
if retriever.index and retriever.index.ntotal > 0:
    test_query = "Quelles sont les informations principales ?"
    eval_results = evaluate_retrieval(test_query)

# Cellule 9: Comparaison de différentes questions
def compare_queries(queries: list):
    """
    Compare les performances pour différentes questions
    """
    import pandas as pd
    
    comparison = []
    
    for query in queries:
        results = retriever.search(query, top_k=5)
        avg_score = sum(r['score'] for r in results) / len(results)
        max_score = max(r['score'] for r in results)
        
        comparison.append({
            'Question': query[:50] + "...",
            'Score moyen': f"{avg_score:.4f}",
            'Score max': f"{max_score:.4f}",
            'Nb résultats': len(results)
        })
    
    df = pd.DataFrame(comparison)
    return df

# Test de comparaison
if retriever.index and retriever.index.ntotal > 0:
    test_queries = [
        "Quel est le contexte ?",
        "Quels sont les objectifs ?",
        "Quelles sont les méthodes utilisées ?",
        "Quels sont les résultats obtenus ?"
    ]
    
    comparison_df = compare_queries(test_queries)
    print("\n📊 Comparaison des requêtes:")
    display(comparison_df)

# Cellule 10: Export des résultats
import json
from datetime import datetime

def export_test_results(question: str, response: dict, output_dir: Path = None):
    """
    Exporte les résultats d'un test
    """
    output_dir = output_dir or config.DATA_DIR / "test_results"
    output_dir.mkdir(exist_ok=True)
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_file = output_dir / f"test_{timestamp}.json"
    
    test_data = {
        'timestamp': timestamp,
        'question': question,
        'answer': response['answer'],
        'sources': response['sources'],
        'context_used': response['context_used']
    }
    
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(test_data, f, ensure_ascii=False, indent=2)
    
    print(f"✅ Résultats exportés: {output_file}")

# Exemple d'export
if 'response' in locals():
    export_test_results(questions[0], response)